In [ ]:
import pandas as pd
import numpy as np
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score
import warnings
warnings.filterwarnings("ignore")

/usr/local/lib/python3.7/dist-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)


In [ ]:
from google.colab import files
uploaded = files.upload()

Saving test.csv to test.csv
Saving train.csv to train.csv


In [ ]:

train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
train_original=train.copy()
test_original=test.copy()

In [ ]:
test.head()

,Id,income,age,experience,married,house_ownership,car_ownership,profession,city,state,current_job_years,current_house_years
0,144036,96682,52,16,married,rented,no,Petroleum_Engineer,Mehsana,Gujarat,11,12
1,166701,7478707,39,9,single,rented,no,Design_Engineer,Ranchi,Jharkhand,3,10
2,238331,6018693,53,12,single,rented,no,Army_officer,Bhiwandi,Maharashtra,5,11
3,144552,5282884,72,3,single,rented,no,Politician,Munger,Bihar,3,10
4,203146,9312068,23,0,single,rented,no,Industrial_Engineer,Panipat,Haryana,0,11


In [ ]:
train=train.drop(columns = ['Id', 'city', 'state', 'profession', 'current_house_years'],axis=1)
test=test.drop(columns = ['Id', 'city', 'state', 'profession', 'current_house_years'],axis=1)


In [ ]:
X = train.drop('risk_flag',1)
y = train.risk_flag

In [ ]:
X = pd.get_dummies(X)
train=pd.get_dummies(train)
test=pd.get_dummies(test)
X.head()


,income,age,experience,current_job_years,married_married,married_single,house_ownership_norent_noown,house_ownership_owned,house_ownership_rented,car_ownership_no,car_ownership_yes
0,2074419,37,1,1,0,1,0,0,1,1,0
1,3728026,48,17,11,0,1,0,0,1,0,1
2,7159687,38,5,3,0,1,0,0,1,1,0
3,277326,45,9,9,0,1,0,0,1,1,0
4,8822191,26,13,3,0,1,0,0,1,1,0


In [ ]:
smt= SMOTE()
c1=list(X.columns.values)
X, y = smt.fit_resample(X, y)
X = pd.DataFrame(X, columns = c1)
X.head()

,income,age,experience,current_job_years,married_married,married_single,house_ownership_norent_noown,house_ownership_owned,house_ownership_rented,car_ownership_no,car_ownership_yes
0,2074419,37,1,1,0,1,0,0,1,1,0
1,3728026,48,17,11,0,1,0,0,1,0,1
2,7159687,38,5,3,0,1,0,0,1,1,0
3,277326,45,9,9,0,1,0,0,1,1,0
4,8822191,26,13,3,0,1,0,0,1,1,0


In [ ]:
paramgrid = {'max_depth': list(range(2,32,2)), 'n_estimators': list(range(10,100,10))}
grid_search=GridSearchCV(RandomForestClassifier(random_state=1),paramgrid)
x_train, x_cv, y_train, y_cv = train_test_split(X,y, test_size=0.15, random_state=1)
grid_search.fit(x_train,y_train)
GridSearchCV(estimator=RandomForestClassifier(random_state=None), param_grid={'max_depth': [2, 4, 6, 8, 10, 12, 14, 16, 18, 20, 22, 24, 26, 28, 30, 32], 'n_estimators': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100]})
grid_search.best_estimator_
RandomForestClassifier(n_estimators = 100, max_depth=32, random_state=None, max_features=None)
i=1
mean = 0
kf = StratifiedKFold(n_splits=100, random_state=1, shuffle=True)
for train_index,test_index in kf.split(X,y):
    print ('\n{} of kfold {} '.format(i,kf.n_splits))
    xtr,xvl = X.loc[train_index],X.loc[test_index]
    ytr,yvl = y[train_index],y[test_index]
    model = RandomForestClassifier(n_estimators = 100, random_state=None, max_depth=32, max_features=None)
    model.fit(xtr,ytr)
    pred_test = model.predict(xvl)
    score = accuracy_score(yvl,pred_test)
    mean += score
    print ('accuracy_score',score)
    i+=1
    pred_test = model.predict(test)
    pred = model.predict_proba(xvl)[:,1]
print ('\n Mean Validation Accuracy',mean/(i-1))

In [ ]:
sub = {'risk_flag':1, 'Id':2}
sub['risk_flag']=pred_test
sub['Id']=test_original['Id']
pd.DataFrame(sub, columns=['Id', 'risk_flag']).to_csv('grid.csv')